## Домашнее задание 11

1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для того же корпуса что вы выбрали из предыдущего дз)
2. Скачиваем любой новостной датасет из https://github.com/natasha/corus/blob/master/README.md
    - реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу)
    - визуально оценить качество

## Решение

1. Установим и импортируем необходимые библиотеки

In [1]:
!pip install transformers annoy

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import unicodedata
import re
import random
import transformers
import os
import io
import time
import torch
import annoy
import tqdm

from sklearn.model_selection import train_test_split
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

Снова загрузим польско-английский датасет и разделим его на польскую и английскую части. Всю предобработку оставим для предобученной модели.

In [3]:
!wget http://www.manythings.org/anki/pol-eng.zip

!rm -rf pol-eng
!mkdir pol-eng
!unzip pol-eng.zip -d pol-eng/

--2022-09-18 15:40:11--  http://www.manythings.org/anki/pol-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1843409 (1.8M) [application/zip]
Saving to: ‘pol-eng.zip’

pol-eng.zip         100%[===================>]   1.76M  6.81MB/s    in 0.3s    

2022-09-18 15:40:11 (6.81 MB/s) - ‘pol-eng.zip’ saved [1843409/1843409]

Archive:  pol-eng.zip
  inflating: pol-eng/pol.txt         
  inflating: pol-eng/_about.txt      


In [4]:
path_to_file = "/kaggle/working/pol-eng/pol.txt"

def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[w for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

en, pol = create_dataset(path_to_file, None)
print(en[0])
print(pol[0])

Go.
Idź.


Загрузим предобученную модель для перевода с польского на английский с `HuggingFace`.

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-pl-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-pl-en")

# Пример перевода
inputs = tokenizer.encode('Byłem w sobotę na zakupach.', return_tensors='pt')
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]).replace('<pad> ', ''))

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/815k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/767k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295M [00:00<?, ?B/s]

I went shopping on Saturday.


Отберем случайно 30 предложений из датасета и переведем их с помощью предобученной модели.

In [6]:
ids = random.choices(range(len(pol)), k=30)

input_texts = []
target_texts = []
prediction_texts = []

for id in ids:
    input_texts.append(pol[id])
    target_texts.append(en[id])
    
    inputs = tokenizer.encode(pol[id], return_tensors='pt')
    outputs = model.generate(inputs)
    prediction_texts.append(tokenizer.decode(outputs[0]).replace('<pad> ', ''))

In [7]:
df = pd.DataFrame(list(zip(input_texts, prediction_texts, target_texts)), 
                  columns=['Input_text', 'Prediction_text', 'Target_text'])

pd.set_option('display.max_colwidth', None)
display(df)

,Input_text,Prediction_text,Target_text
0,Po zjedzeniu obiadu była gotowa do wyjścia.,"After lunch, she was ready to go.","After she had lunch, she got ready to go out."
1,Okazują sobie miłość uściskami.,They show their love with a hug.,They're expressing their love by hugging.
2,Jest jakaś nadzieja na jego sukces?,Is there any hope for his success?,Is there any hope of his success?
3,"Idę do baru na drinka i proponuję, żebyś zrobił to samo.",I'm going to the bar for a drink and I suggest you do the same.,"I'm going up to the bar for a drink, and I suggest you do the same."
4,Napisz tutaj swój adres.,Write your address here.,Write your address here.
5,Powinniśmy użyć kominka dzisiejszej nocy.,We should use the fireplace tonight.,We should use the fireplace tonight.
6,"Jeśli twoja firma prowadzi interesy przede wszystkim ze Stanami, musisz uczyć się angielskiego z rodowitym Amerykaninem.","If your company does business with the U.S. first, you have to learn English with a native American.","If your company primarily does business with America, then you should be studying English with a native speaker from America."
7,Tom planował kogoś zabić.,Tom was planning on killing someone.,Tom was planning on killing someone.
8,"Ona może rozwiązać ten problem, ja zresztą też.","She can solve this problem, and so can I.","She could solve the problem, and so could I."
9,Jestem teraz bardzo głodny.,I'm really hungry right now.,I'm very hungry now.


Невооруженным глазом видно, что качество перевода значительно улучшилось по сравнению с моделями с предыдущего занятия. Даже в тех случаях, где предсказанный перевод не совпадает с таргетом идеально он отличается минимально (время, синонимы) и несильно влияет на смысл. 

2. Датасет со статьями с "Газета.ру" не доступен для скачивания по ссылке, указанной на гитхабе Corus https://github.com/natasha/corus/blob/master/README.md#load_ods_gazeta. Загрузим его по актуальной ссылке и сформируем датасет. 

In [8]:
!wget https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz

--2022-09-18 15:40:58--  https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz [following]
--2022-09-18 15:40:59--  https://www.dropbox.com/s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4e743b2cbaeea453594c028548.dl.dropboxusercontent.com/cd/0/inline/BtMwfmurnIDFgkWp7CifIuFekcentCkV7ZigbB1KeBfIiCh4UOLHRsKIpzOUZ-4-xaF9BKoN8zY2vgtE2a48EvxuFyDGyJNKkKtMishMBWSyBt1vbWITto5WJcgO9_BnM7c_UpN-dNBhRkH8yEFZt0cqsm6JlMtreiKnokK1fgALYw/file# [following]
--2022-09-18 15:40:59--  https://uc4e743b2cbaeea453594c028548.dl.dropboxusercontent.com/cd/0/inline/BtMwfmurnIDFgkWp7CifIuFekcentCkV7ZigbB1KeBfIiCh4UOLHRsKIpzO

In [9]:
!tar -xzvf gazeta_jsonl.tar.gz

gazeta_train.jsonl
gazeta_test.jsonl
gazeta_val.jsonl


In [10]:
df_train = pd.read_json(path_or_buf='gazeta_train.jsonl', lines=True)
df_val = pd.read_json(path_or_buf='gazeta_val.jsonl', lines=True)
df_test = pd.read_json(path_or_buf='gazeta_test.jsonl', lines=True)

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
df.sample()

url  \
4736  https://www.gazeta.ru/sport/2011/05/a_3622545.shtml   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [11]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
def get_norm_vector(text):
    tokens = tokenizer(text, return_token_type_ids=False, return_tensors='pt')
    vector = model(**tokens)
    norm_vector = torch.sum(vector[0][0], 0) / len(vector[0][0])
    return norm_vector

def get_response(question, index, index_map, nn=5):
    answers = index.get_nns_by_vector(get_norm_vector(question), nn)
    return [index_map[i] for i in answers]

In [13]:
vector_size = 768

deeppavlov_index = annoy.AnnoyIndex(vector_size, 'angular')
index_map = {}
counter = 0

# Для ускорения процесса рассмотрим только первые 15000 статей
for i in tqdm.tqdm(range(15000)):
    index_map[i] = df['text'][i]
    deeppavlov_index.add_item(i, get_norm_vector(df['summary'][i]))

deeppavlov_index.build(10)

100%|██████████| 15000/15000 [42:10<00:00,  5.93it/s] 


True

Выведем пять ближайших статей для следующего запроса.

In [14]:
TEXT = "Футбольные фанаты устроили беспорядки"

In [15]:
n_texts = pd.DataFrame(get_response(TEXT, deeppavlov_index, index_map), 
                       columns=['text'])
n_texts.head()

text
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Во вторник в Каире начался суд по делу о массовых беспорядках после футбольного матча в Порт-Саиде между местной командой «Аль-Масри» и каирской «Аль-Ахли». Жертвами побоища тогда стали 74 человека, а еще около 300 получили травмы. На скамье подсудимых оказались 73 обвиняемых, в том числе трое представителей футбольного клуба «Аль-Марси» и девять полицейских (среди которых генерал-майор Иссам Самак, который отвечал за безопасность во время злополучного матча). Гражданские подсудимые обвиняются в убийстве по неосторожности и насилии, полицейским инкриминируется ненадлежащее исполнение служебного долга. Фанаты не признают себя виновными. Суд по делу о побоище проходит в том же зале полицейской академии Каира, где в прошлом году судили бывшего президента страны Хосни Мубарака (и чье имя она носила до народного восстания в Египте), сообщает «Би-би-си». Проводить процесс на окраине столицы Египта на этот раз также было решено из соображений безопасности. Тем не менее заседание суда началось с беспорядков, которые прямо в клетке устроили подсудимые, передает Reuters. Прокурор начал зачитывать обвинительное заключение, когда десятки фигурантов дела, одетых в белое, запрыгнули на скамейки, закричали «Мы добьемся правосудия или убьем себя» и замахали кулаками в сторону представителей прокуратуры. После этого одни родственники жертв разрыдались, другие выше подняли над головами фотографии своих погибших близких. Судья Абдель Магид Махмуд вышел из зала на время, пока все успокоится. После того как подсудимых утихом

Нетрудно заметить, что практически все статьи из предложенных моделью тем или иным образом связаны с заданным запросом, что может указывать на довольно хорошее качество.